In [2]:
from ia_model.data_preparation import load_data, prepare_data, split_data
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, r2_score
from itertools import product
import numpy as np
import pandas as pd

In [4]:
# Configuration parameters for the model and data preparation
FILE_PATH = 'aggregated_data4.csv' #This data can be download from https://rcavieses.pythonanywhere.com/
FEATURE_COL = ['NOMBRE PRINCIPAL','NOMBRE ESTADO','NOMBRE OFICINA','SST','PESO DESEMBARCADO_KILOGRAMOS']#Change this if you want to add input variables
TARGET_COL = 'PESO DESEMBARCADO_KILOGRAMOS'
CATEGORICAL_COL = ['NOMBRE PRINCIPAL','NOMBRE ESTADO','NOMBRE OFICINA']
NUMERIC_COL = ['PESO DESEMBARCADO_KILOGRAMOS','SST']
steps_forecast = 12
time_steps= 12 # number of steps that lstm forecast, change this will be errors on scoore metrics ;)
neurons_list = list(range(10, 16))
layers_list = [3, 4]
activations_list = ['relu', 'sigmoid']
# activations_list = ['relu', 'tanh', 'sigmoid']
EPOCHS = 50
BATCH_SIZE = 32
MODEL_PATH = 'mi_modelo.h5'
SCALER_X_PATH = 'scaler_X.pkl'
SCALER_Y_PATH = 'scaler_Y.pkl'

In [5]:
df = pd.read_csv('aggregated_data4.csv')
X, y, scaler_x,scaler_y, encoder , n_features = prepare_data(df, TARGET_COL, time_steps, CATEGORICAL_COL,NUMERIC_COL)
X_train, X_test, y_train, y_test = split_data(X, y)

In [6]:
def build_dynamic_model(time_steps, n_features, layer_configs, dropout=0.2):
    model = Sequential()
    for i, config in enumerate(layer_configs):
        if config['type'] == 'LSTM':
            # For the first LSTM layer, specify input_shape
            if i == 0:
                model.add(LSTM(config['neurons'], activation=config['activation'], input_shape=(time_steps, n_features), return_sequences=(i < len(layer_configs) - 1)))
            else:
                model.add(LSTM(config['neurons'], activation=config['activation'], return_sequences=(i < len(layer_configs) - 1)))
        elif config['type'] == 'Dense':
            model.add(Dense(config['neurons'], activation= 'linear'))
        if i < len(layer_configs) - 1:  # Add dropout except for the last layer
            model.add(Dropout(dropout))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


def train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=50, batch_size=32):
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Fit the model using the reshaped targets
    # model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1, callbacks=[early_stopping])
    predictions = model.predict(X_test)

    # Adjust the shape of predictions and y_test if necessary before calculating MSE and R^2
    mse = mean_squared_error(y_test.reshape(-1), predictions.reshape(-1))
    r2 = r2_score(y_test.reshape(-1), predictions.reshape(-1))
    return mse, r2

def generate_configurations(neurons_list, layers_list, activations_list):
    configurations = []
    for layers, activation in product(layers_list, activations_list):
        for neurons in product(neurons_list, repeat=layers):
            config = [{'type': 'LSTM', 'neurons': n, 'activation': activation} for n in neurons[:-1]]
            config.append({'type': 'Dense', 'neurons': neurons[-1], 'activation': 'sigmoid'})  # Output layer always Dense (reflecting a common practice for binary classification or regression tasks where the output is scaled between 0 and 1)
            configurations.append(config)
    return configurations

def find_best_model(X_train, y_train, X_test, y_test, configurations, epochs=50, batch_size=32):
    best_mse = float('inf')
    best_r2 = float('-inf')
    best_config = None
    for config in configurations:
        model = build_dynamic_model(X_train.shape[1], X_train.shape[2], config)
        # mse, r2 = train_and_evaluate(model, X_train, y_train, X_test, y_test, EPOCHS, BATCH_SIZE)
        mse, r2 = train_and_evaluate(model, X_train, y_train, X_test, y_test, epochs=epochs, batch_size=batch_size)
        if mse < best_mse or (mse == best_mse and r2 > best_r2):
            best_mse, best_r2, best_config = mse, r2, config
    return best_config, best_mse, best_r2

# # Define the parameter ranges
# neurons_list = list(range(10, 21))
# layers_list = [2, 3, 4, 5]
# activations_list = ['relu', 'tanh', 'sigmoid']

configurations = generate_configurations(neurons_list, layers_list, activations_list)

# Assuming X_train, y_train, X_test, y_test are defined
best_config, best_mse, best_r2 = find_best_model(X_train, y_train, X_test, y_test, configurations)
print(f"Best configuration: {best_config}\nMSE: {best_mse}, R^2: {best_r2}")



Epoch 1/50



InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/mean_squared_error/BroadcastGradientArgs defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Users\ricar\anaconda3\envs\pro24\Lib\asyncio\events.py", line 80, in _run

  